In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import sklearn
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sklearn import feature_extraction,model_selection
from sklearn.metrics import confusion_matrix
import glob
import os
import string
import tensorflow as tf
import keras as K
import zipfile
from itertools import chain
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using TensorFlow backend.


In [ ]:
# importing Word2Vec 
from gensim.models import Word2Vec
from gensim.models import FastText

In [ ]:
X=pd.read_csv('/content/drive/My Drive/Hindi_Combined.csv')
X.head()

,Unnamed: 0,Text,Label
0,0,RANDTV tumhare najayaz baap is area hai ki wah...,2
1,1,First stage par dus jootey khaye Grover se,2
2,2,Salman aur aamir ki kounsi movie release huyee...,2
3,3,Wk aur bhakt aa gya. Abe americans ka bolna h...,1
4,4,Beta to tu apne baap ka hai ...permission to t...,0


In [ ]:
df1=pd.read_csv('/content/drive/My Drive/Colab Notebooks/hin_dev.csv',index_col='ID')
df1.head()

,Text,Sub-task A,Sub-task B
ID,,,
C38.9,bkl interviewers kuch jaada hi open minded bnt...,OAG,NGEN
C4.1510,Bhaiya shaadi mein zaroor aana movie ka plot j...,NAG,NGEN
C19.95,Section 375 hai kya??? .... Ye to batate kam s...,NAG,NGEN
C4.281,कबीर सिंह hit Hui इससे पता चलता है आजकल के लोग...,OAG,NGEN
C4.216,Maine itni kam dislike kbhi nhii dekhi,NAG,NGEN


In [ ]:
Y=df1.drop(columns={'Sub-task B'})
type(Y['Sub-task A'])

pandas.core.series.Series

In [ ]:
X.head()

,Unnamed: 0,Text,Label
0,0,RANDTV tumhare najayaz baap is area hai ki wah...,2
1,1,First stage par dus jootey khaye Grover se,2
2,2,Salman aur aamir ki kounsi movie release huyee...,2
3,3,Wk aur bhakt aa gya. Abe americans ka bolna h...,1
4,4,Beta to tu apne baap ka hai ...permission to t...,0


In [ ]:
Y['Sub-task A']=Y['Sub-task A'].map({'OAG':2,'CAG':1,'NAG':0})
Y.head()

,Text,Sub-task A
ID,,
C38.9,bkl interviewers kuch jaada hi open minded bnt...,2
C4.1510,Bhaiya shaadi mein zaroor aana movie ka plot j...,0
C19.95,Section 375 hai kya??? .... Ye to batate kam s...,0
C4.281,कबीर सिंह hit Hui इससे पता चलता है आजकल के लोग...,2
C4.216,Maine itni kam dislike kbhi nhii dekhi,0


In [ ]:
X_train=X['Text']
X_train

0       RANDTV tumhare najayaz baap is area hai ki wah...
1             First stage par dus jootey khaye Grover  se
2       Salman aur aamir ki kounsi movie release huyee...
3       Wk aur bhakt aa gya.  Abe americans ka bolna h...
4       Beta to tu apne baap ka hai ...permission to t...
                              ...                        
9144    Asexual h.. bisexual... homosexual... bhai ase...
9145    Video pura dekne ke pahile hi mai bhai ke vide...
9146                     konsa place hai bhai ...nam bolo
9147                             Kuch zada hi likh diya 🙄
9148    scary tube par kya woh karna sahi tha? Pucha k...
Name: Text, Length: 9149, dtype: object

In [ ]:
y_train=X['Label']
y_train

0       2
1       2
2       2
3       1
4       0
       ..
9144    0
9145    0
9146    0
9147    0
9148    0
Name: Label, Length: 9149, dtype: int64

In [ ]:
X_test=Y['Text']
X_test

ID
C38.9        bkl interviewers kuch jaada hi open minded bnt...
C4.1510      Bhaiya shaadi mein zaroor aana movie ka plot j...
C19.95       Section 375 hai kya??? .... Ye to batate kam s...
C4.281       कबीर सिंह hit Hui इससे पता चलता है आजकल के लोग...
C4.216                  Maine itni kam dislike kbhi nhii dekhi
                                   ...                        
C29.18                   Are bhai konsi duniya mai rehate ho😂😂
C9.225                                            Sahi to bola
C7.1685                                Or tum Kay Kar raya ho?
C36.692.1    Ye bhaat sahi hain par zyada dhin thine ga nah...
C4.1202                   Islye maine Kabaad singh nai dekha..
Name: Text, Length: 997, dtype: object

In [ ]:
y_test=Y['Sub-task A']
y_test

ID
C38.9        2
C4.1510      0
C19.95       0
C4.281       2
C4.216       0
            ..
C29.18       0
C9.225       0
C7.1685      0
C36.692.1    0
C4.1202      0
Name: Sub-task A, Length: 997, dtype: int64

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((9149,), (997,), (9149,), (997,))

In [ ]:
def create_tokens(lines):
  #stop_words = set(nltk.corpus.stopwords.words('english'))
  t_line=lines.apply(lambda x: nltk.word_tokenize(x))
  #t_line=t_line.apply(lambda x: [w for w in x if not w in stop_words])
  return t_line

In [ ]:
X_train=create_tokens(X_train)
print(X_train)
tokenizer= Tokenizer()
tokenizer.fit_on_texts(X_train)
tokenizer


0       [RANDTV, tumhare, najayaz, baap, is, area, hai...
1       [First, stage, par, dus, jootey, khaye, Grover...
2       [Salman, aur, aamir, ki, kounsi, movie, releas...
3       [Wk, aur, bhakt, aa, gya, ., Abe, americans, k...
4       [Beta, to, tu, apne, baap, ka, hai, ..., permi...
                              ...                        
9144    [Asexual, h.., bisexual, ..., homosexual, ...,...
9145    [Video, pura, dekne, ke, pahile, hi, mai, bhai...
9146            [konsa, place, hai, bhai, ..., nam, bolo]
9147                      [Kuch, zada, hi, likh, diya, 🙄]
9148    [scary, tube, par, kya, woh, karna, sahi, tha,...
Name: Text, Length: 9149, dtype: object


In [ ]:
length=np.average(X_train.apply(lambda x:len(x)*len(x)))
length=np.sqrt(length)
length=48

In [ ]:
def encode_text(tokenizer, lines, length):
  encoded = tokenizer.texts_to_sequences(lines)
  padded = pad_sequences(encoded, maxlen=length, padding='post')
  return padded

In [ ]:
word_index=tokenizer.word_index

In [ ]:
X_test=create_tokens(X_test)
print(X_test)

ID
C38.9        [bkl, interviewers, kuch, jaada, hi, open, min...
C4.1510      [Bhaiya, shaadi, mein, zaroor, aana, movie, ka...
C19.95       [Section, 375, hai, kya, ?, ?, ?, ..., ., Ye, ...
C4.281       [कबीर, सिंह, hit, Hui, इससे, पता, चलता, है, आज...
C4.216          [Maine, itni, kam, dislike, kbhi, nhii, dekhi]
                                   ...                        
C29.18           [Are, bhai, konsi, duniya, mai, rehate, ho😂😂]
C9.225                                        [Sahi, to, bola]
C7.1685                       [Or, tum, Kay, Kar, raya, ho, ?]
C36.692.1    [Ye, bhaat, sahi, hain, par, zyada, dhin, thin...
C4.1202            [Islye, maine, Kabaad, singh, nai, dekha..]
Name: Text, Length: 997, dtype: object


In [ ]:
X_train=encode_text(tokenizer,X_train,length)
X_train
#pd.DataFrame(train_x).to_csv("train_file.csv")

array([[12745,   589,  8238, ...,     0,     0,     0],
       [ 2590,  6237,    83, ...,     0,     0,     0],
       [ 1480,    28,  6238, ...,     0,     0,     0],
       ...,
       [ 3946,  5475,     2, ...,     0,     0,     0],
       [   59,  4754,    29, ...,     0,     0,     0],
       [12471,  6207,    83, ...,     0,     0,     0]], dtype=int32)

In [ ]:
X_test=encode_text(tokenizer,X_test,length)
X_test
#pd.DataFrame(train_x).to_csv("train_file.csv")

array([[  59, 7996,   29, ...,    0,    0,    0],
       [ 843, 2618,   79, ...,    0,    0,    0],
       [1637, 1345,    2, ...,    0,    0,    0],
       ...,
       [  47,   85,  926, ...,    0,    0,    0],
       [  26, 6231,   82, ...,    0,    0,    0],
       [8248,  408,  125, ...,    0,    0,    0]], dtype=int32)

In [ ]:
from keras.utils.np_utils import to_categorical
y_test = to_categorical(np.asarray(y_test))
y_train = to_categorical(np.asarray(y_train))
print(y_test)
print(y_train)

[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [ ]:
# print(len(X_train))
# print(len(X_test))
# print(len(y_train))
# print(len(y_test))
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((9149, 48), (997, 48), (9149, 3), (997, 3))

In [ ]:
# Creating embedding matrix
# Using 
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/crawl-300d-2M.vec.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
#load pre-trained embedding:
embeddings_index = {}
f = open('/content/crawl-300d-2M.vec', encoding="utf8")
for line in f:
    values = line.split()#This splits each values in a  line with spaces
    word = values[0] # This store the first values which is the word in english
    coefs = np.asarray(values[1:], dtype='float32') # This converts all splitted values from 1 to last(expect first word) into an array
    embeddings_index[word] = coefs # english word = array of numerical values
f.close()
print('Total %s word vectors in crawl-300-2M.' % len(embeddings_index))

Total 1999996 word vectors in crawl-300-2M.


In [ ]:
print((X_train))
print((X_test))
print((y_train))
print((y_test))

[[12745   589  8238 ...     0     0     0]
 [ 2590  6237    83 ...     0     0     0]
 [ 1480    28  6238 ...     0     0     0]
 ...
 [ 3946  5475     2 ...     0     0     0]
 [   59  4754    29 ...     0     0     0]
 [12471  6207    83 ...     0     0     0]]
[[  59 7996   29 ...    0    0    0]
 [ 843 2618   79 ...    0    0    0]
 [1637 1345    2 ...    0    0    0]
 ...
 [  47   85  926 ...    0    0    0]
 [  26 6231   82 ...    0    0    0]
 [8248  408  125 ...    0    0    0]]
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [ ]:
# Initialize the weight for each word for the embedding layer. Words not present
# in the dictionary initialized by random value 
EMBEDDING_DIM = 300
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
#embedding_matrix = (16433, 100) NOTE: It is from uniques words in our tweets
for word, i in word_index.items(): # Selecting word from our tweet unique words list for changing with the values from glove
    embedding_vector = embeddings_index.get(word) # Finding our tweet words in glove and taking its array of 100 values
    if embedding_vector is not None: 
#words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
embedding_matrix.shape

(33453, 300)

In [ ]:

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.utils.vis_utils import plot_model
model = Sequential()
model.add(Embedding(input_dim = 33453, output_dim = 300, weights=[embedding_matrix], input_length=length,trainable=False))
model.add(Conv1D(filters = 32,kernel_size=3, activation='tanh'))
model.add(MaxPooling1D(3))
model.add(Dropout(0.3))
model.add(Conv1D(filters = 64,kernel_size=3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation = 'tanh'))
model.add(Dropout(0.3))
model.add(Dense(3, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 48, 300)           10035900  
_________________________________________________________________
conv1d_28 (Conv1D)           (None, 46, 32)            28832     
_________________________________________________________________
max_pooling1d_28 (MaxPooling (None, 15, 32)            0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 15, 32)            0         
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 13, 64)            6208      
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 4, 64)             0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 4, 64)           

In [ ]:
model.fit(X_train, y_train, validation_data = (X_test, y_test),epochs=500,batch_size=128, verbose=2)

Train on 9149 samples, validate on 997 samples
Epoch 1/500
 - 2s - loss: 1.1264 - acc: 0.3789 - val_loss: 1.0237 - val_acc: 0.5486
Epoch 2/500
 - 0s - loss: 1.0722 - acc: 0.4143 - val_loss: 1.0077 - val_acc: 0.5436
Epoch 3/500
 - 0s - loss: 1.0636 - acc: 0.4258 - val_loss: 0.9877 - val_acc: 0.5105
Epoch 4/500
 - 0s - loss: 1.0559 - acc: 0.4279 - val_loss: 0.9925 - val_acc: 0.5527
Epoch 5/500
 - 0s - loss: 1.0423 - acc: 0.4524 - val_loss: 0.9711 - val_acc: 0.5496
Epoch 6/500
 - 0s - loss: 1.0203 - acc: 0.4638 - val_loss: 0.9351 - val_acc: 0.5948
Epoch 7/500
 - 0s - loss: 0.9988 - acc: 0.4884 - val_loss: 0.9223 - val_acc: 0.5968
Epoch 8/500
 - 0s - loss: 0.9876 - acc: 0.4901 - val_loss: 0.8833 - val_acc: 0.6048
Epoch 9/500
 - 0s - loss: 0.9692 - acc: 0.5053 - val_loss: 0.8831 - val_acc: 0.6169
Epoch 10/500
 - 0s - loss: 0.9619 - acc: 0.5104 - val_loss: 0.8892 - val_acc: 0.5988
Epoch 11/500
 - 0s - loss: 0.9418 - acc: 0.5250 - val_loss: 0.8517 - val_acc: 0.5998
Epoch 12/500
 - 0s - loss: 

KeyboardInterrupt: ignored

In [ ]:
predictions_test = model.predict(X_test)
predictions_test

array([[1.9961235e-01, 5.0446606e-01, 2.9592162e-01],
       [9.8950571e-01, 1.0011367e-02, 4.8290304e-04],
       [9.5669359e-01, 2.5278265e-02, 1.8028177e-02],
       ...,
       [1.0830624e-02, 4.7718119e-02, 9.4145131e-01],
       [4.3151677e-01, 5.6308556e-01, 5.3976364e-03],
       [9.1787487e-01, 2.2921160e-02, 5.9204035e-02]], dtype=float32)

In [ ]:
pred_test=np.zeros_like(predictions_test)
pred_test[np.arange(len(predictions_test)),predictions_test.argmax(1)]=1
output_DataFrame=pd.DataFrame(pred_test.argmax(axis=1),index=df1.index)
output_DataFrame[0]=output_DataFrame[0].map({0:'NAG',1:'CAG',2:'OAG'})
output_DataFrame.to_csv("Hindi Subtask B ou.csv")

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred_test))

              precision    recall  f1-score   support

           0       0.79      0.72      0.75       578
           1       0.38      0.48      0.42       211
           2       0.50      0.48      0.49       208

   micro avg       0.62      0.62      0.62       997
   macro avg       0.55      0.56      0.55       997
weighted avg       0.64      0.62      0.63       997
 samples avg       0.62      0.62      0.62       997



In [ ]:
df_test=pd.read_csv("/content/drive/My Drive/Colab Notebooks/hin_test.csv")
df_test.head()

,ID,Text
0,C52.17,ko
1,C52.39,ladkiyon video
2,C52.73,ki video gahrep
3,C60.3,o sadharon video bhai
4,C60.43,ba bhai kyea bola tum moza aaa giea 😌😌😌😂😂😂


In [ ]:
model.save('HindiTaskB.h5')

In [ ]:
test_text=encode_text(tokenizer,df_test['Text'],length)
test_text

array([[   7,    0,    0, ...,    0,    0,    0],
       [3229,  126,    0, ...,    0,    0,    0],
       [   6,  126,    0, ...,    0,    0,    0],
       ...,
       [ 271, 1039, 8299, ...,  159,  869, 4289],
       [1574, 2566,   34, ..., 4018, 4567,  311],
       [3443,  554,  168, ...,    0,    0,    0]], dtype=int32)

In [ ]:
predictions_test = model.predict(test_text)

predictions_test

array([[8.7135029e-01, 2.9158773e-02, 9.9490970e-02],
       [9.8230928e-01, 2.9442701e-03, 1.4746461e-02],
       [9.7268361e-01, 1.0196120e-02, 1.7120333e-02],
       ...,
       [7.1363673e-02, 5.2094120e-03, 9.2342693e-01],
       [7.7121731e-06, 5.6996714e-07, 9.9999177e-01],
       [9.5991891e-07, 5.2664673e-05, 9.9994636e-01]], dtype=float32)

In [ ]:
pred_test=np.zeros_like(predictions_test)
pred_test[np.arange(len(predictions_test)),predictions_test.argmax(1)]=1
output_DataFrame=pd.DataFrame(pred_test.argmax(axis=1),index=df_test['ID'])
output_DataFrame[0]=output_DataFrame[0].map({0:'NAG',1:'CAG',2:'OAG'})
output_DataFrame.to_csv("Hindi Subtask B CNN Fasttext output.csv")